### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

### Configure Settings

In [ ]:
pd.set_option('display.max_columns', None)
%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
sns.set_style('whitegrid')
plt.rcParams.update({'figure.dpi': 120})

### Load datasets

In [ ]:
sales = pd.read_csv('../data/sales_data.csv')
products = pd.read_csv('../data/product_info.csv')
customers = pd.read_csv('../data/customer_info.csv')

print(f"Sales:, {sales.shape}")
print(f"Products:, {products.shape}")
print(f"Customers:, {customers.shape}")

display(sales)
display(products)
display(customers)

### Data Exploration

In [ ]:
for df, name in [(sales, 'sales'), (products, 'products'), (customers, 'customers')]:
    print("=== ", name, " ===")
    display(df.info())
    display(df.describe(include='all'))

### Data Cleaning

#### Trim whitespace everywhere

In [ ]:
def strip_all_str_cols(df):
    for c in df.select_dtypes(include='object').columns:
        df[c] = df[c].astype(str).str.strip()
        df.loc[df[c].isin(['nan', 'None', 'NoneType']), c] = np.nan
    return df

sales = strip_all_str_cols(sales)
products = strip_all_str_cols(products)
customers = strip_all_str_cols(customers)

#### Fix numeric columns in sales: quantity, unit_price, discount_applied

In [ ]:
sales['quantity'] = sales['quantity'].replace({'three': '3', 'five': '5'}).astype(object)

sales['quantity'] = pd.to_numeric(sales['quantity'], errors='coerce')
sales['unit_price'] = pd.to_numeric(sales['unit_price'], errors='coerce')
sales['discount_applied'] = pd.to_numeric(sales['discount_applied'], errors='coerce')

sales['quantity_imputed_flag'] = sales['quantity'].isnull()

sales.loc[sales['quantity'].isnull(), 'quantity'] = 1
sales['quantity'] = sales['quantity'].astype(int)

#### Dates parsing

In [ ]:
sales['order_date'] = pd.to_datetime(
    sales['order_date'], 
    format='%d-%m-%Y', 
    errors='coerce'
)

products['launch_date'] = pd.to_datetime(
    products['launch_date'], 
    format='%d-%m-%y', 
    errors='coerce'
)

customers['signup_date'] = pd.to_datetime(
    customers['signup_date'], 
    format='%d-%m-%y', 
    errors='coerce'
)

print("order_date parse nulls:", sales['order_date'].isna().sum())
print("launch_date parse nulls:", products['launch_date'].isna().sum())
print("signup_date parse nulls:", customers['signup_date'].isna().sum())

#### Canonicalize and clean text fields

In [ ]:
def clean_delivery(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    if 'deliver' in s: return 'Delivered'
    if 'delay' in s or 'dely' in s: return 'Delayed'
    if 'cancel' in s: return 'Cancelled'
    return 'Other'

sales['delivery_status'] = sales['delivery_status'].apply(clean_delivery)

pm_map = {
    'credit card': 'Credit Card',
    'bank transfr': 'Bank Transfer',
}
sales['payment_method'] = sales['payment_method'].str.strip().str.lower().map(pm_map).fillna(sales['payment_method'].str.title())

sales['region'] = sales['region'].replace({'nrth': 'North'}).str.title()

customers['region'] = customers['region'].replace({'nrth': 'North'}).str.title()

def clean_loyalty(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    if 'gold' in s or s in ('gld',): return 'Gold'
    if 'silver' in s or s in ('sllver',): return 'Silver'
    if 'bronze' in s or s in ('brnze',): return 'Bronze'
    return 'Other'

customers['loyalty_tier'] = customers['loyalty_tier'].apply(clean_loyalty)

def clean_gender(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    if s.startswith('f'): return 'Female'
    if s.startswith('m'): return 'Male'
    return 'Other'

customers['gender'] = customers['gender'].apply(clean_gender)

#### Handle missing product_id / customer_id / discount_applied / email

In [ ]:
sales['product_id'] = sales['product_id'].fillna('UNKNOWN_PRODUCT')
sales['customer_id'] = sales['customer_id'].fillna('UNKNOWN_CUSTOMER')

sales['discount_imputed_flag'] = sales['discount_applied'].isna()
sales['discount_applied'] = sales['discount_applied'].fillna(0.0)

customers['email'] = customers['email'].replace({'nan': np.nan})

#### Handle duplicates

In [ ]:
sales['synthetic_order_key'] = (
    sales['order_id'].astype(str) + '_' +
    sales['customer_id'].astype(str) + '_' +
    sales['product_id'].astype(str)
)

sales['order_id_duplicate_flag'] = sales.duplicated(subset=['order_id'], keep=False)

conflicting_orders = (
    sales.groupby('order_id')
    .agg({'customer_id':'nunique', 'order_date':'nunique'})
    .query('customer_id > 1 or order_date > 1')
)
print("Conflicting order_ids:\n", conflicting_orders)

sales = sales.dropna(subset=['order_id', 'product_id', 'customer_id'])
customers = customers.dropna(subset=['customer_id'])

In [ ]:
missing_sales = sales[sales.isnull().any(axis=1)]
print(f"Sales rows with missing values: {len(missing_sales)}")
display(missing_sales)

missing_customers = customers[customers.isnull().any(axis=1)]
print(f"Customer rows with missing values: {len(missing_customers)}")
display(missing_customers)


missing_products = products[products.isnull().any(axis=1)]
print(f"Product rows with missing values: {len(missing_products)}")
display(missing_products)

sales_dupes_order = sales[sales.duplicated(subset=['order_id'], keep=False)]
print("Duplicate order_ids:", len(sales_dupes_order))
display(sales_dupes_order)

sales_dupes_line = sales[sales.duplicated(subset=['order_id','product_id'], keep=False)]
print("Duplicate order lines:", len(sales_dupes_line))
display(sales_dupes_line)

customers_dupes = customers[customers.duplicated(subset=['customer_id'], keep=False)]
print("Duplicate customer_ids:", len(customers_dupes))
display(customers_dupes)

products_dupes = products[products.duplicated(subset=['product_id'], keep=False)]
print("Duplicate product_ids:", len(products_dupes))
display(products_dupes)

#### Cleanup

In [ ]:
def clean_data(sales: pd.DataFrame, customers: pd.DataFrame, products: pd.DataFrame):
    sales = sales.dropna(subset=['order_id', 'product_id', 'customer_id']).copy()
    customers = customers.dropna(subset=['customer_id']).copy()
    products = products.dropna(subset=['product_id']).copy()

    sales['was_price_imputed'] = sales['unit_price'].isna()
    sales['was_qty_imputed'] = sales['quantity'].isna()
    sales['was_status_imputed'] = sales['delivery_status'].isna()
    sales['was_payment_imputed'] = sales['payment_method'].isna()
    sales['was_order_date_imputed'] = sales['order_date'].isna()
    customers['was_region_imputed'] = customers['region'].isna()
    customers['was_tier_imputed'] = customers['loyalty_tier'].isna()
    customers['was_gender_imputed'] = customers['gender'].isna()
    customers['was_signup_date_imputed'] = customers['signup_date'].isna()

    # Fill unit_price using product base price
    if 'base_price' in products.columns:
        price_map = products.set_index('product_id')['base_price']
        sales['unit_price'] = sales['unit_price'].fillna(sales['product_id'].map(price_map))

    # Fill quantity with median
    if sales['quantity'].isna().any():
        sales['quantity'].fillna(sales['quantity'].median(), inplace=True)

    # Fill delivery_status and payment_method
    if sales['delivery_status'].isna().any():
        sales['delivery_status'] = sales.groupby('region')['delivery_status'].transform(
            lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x.fillna('Unknown')
        )
    sales['payment_method'] = sales['payment_method'].fillna('Unknown')

    if sales['order_date'].isna().any():
        median_order_date = sales['order_date'].median()
        sales['order_date'] = sales['order_date'].fillna(median_order_date)

    # Fill region based on loyalty_tier mode
    if customers['region'].isna().any():
        customers['region'] = customers.groupby('loyalty_tier')['region'].transform(
            lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x.fillna(customers['region'].mode()[0])
        )

    customers['region'] = customers['region'].fillna(customers['region'].mode()[0])

    if 'total_spent' not in customers.columns:
        customers['total_spent'] = np.nan


    spend_map = sales.groupby('customer_id')['unit_price'].sum()
    customers['total_spent'] = customers['total_spent'].fillna(customers['customer_id'].map(spend_map))

    customers['total_spent'] = customers['total_spent'].fillna(0)


    # Assign based on spend thresholds
    tier_values = np.select(
        [
            customers['total_spent'] > 5000,
            customers['total_spent'] > 2000,
            customers['total_spent'] > 500,
        ],
        ['Gold', 'Silver', 'Bronze'],
        default='Bronze'
    )

    customers.loc[customers['loyalty_tier'].isna(), 'loyalty_tier'] = tier_values[customers['loyalty_tier'].isna()]

    customers['loyalty_tier'] = customers['loyalty_tier'].fillna(customers['loyalty_tier'].mode()[0])


    # Gender imputation
    if 'gender' in customers.columns:
        customers['gender'] = customers['gender'].fillna(customers['gender'])

    # Infer from region
    customers['gender'] = customers.groupby('region')['gender'].transform(
        lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
    )

    customers['gender'] = customers['gender'].fillna(customers['gender'].mode()[0])

    # Fill email and signup_date
    customers['email'] = customers['email'].fillna('unknown@sellers.com')

    if customers['signup_date'].isna().any():
        median_signup = customers['signup_date'].median()
        customers['signup_date'] = customers['signup_date'].fillna(median_signup)


    return (
        sales.reset_index(drop=True),
        customers.reset_index(drop=True),
        products.reset_index(drop=True)
    )


sales, customers, products = clean_data(sales, customers, products)

#### Merge datasets

In [ ]:
merged = sales.merge(products.add_prefix('prod_'), left_on='product_id', right_on='prod_product_id', how='left')
merged = merged.merge(customers.add_prefix('cust_'), left_on='customer_id', right_on='cust_customer_id', how='left')

print('Merged shape:', merged.shape)
merged.info()
merged.head()

#### Feature engineering

In [ ]:
# revenue
merged['unit_price'] = merged['unit_price'].astype(float)
merged['revenue'] = merged['quantity'] * merged['unit_price'] * (1 - merged['discount_applied'])

# order_week
merged['order_week'] = merged['order_date'].dt.isocalendar().week

# price_band
merged['price_band'] = pd.cut(merged['unit_price'], bins=[-1, 15, 30, np.inf], labels=['Low', 'Medium', 'High'])

# days_to_order
merged['launch_date'] = merged['prod_launch_date']
merged['days_to_order'] = (merged['order_date'] - merged['launch_date']).dt.days
merged['days_to_order_flag'] = (merged['days_to_order'] < 0) | (merged['days_to_order'] > 365)

# email_domain
merged['email_domain'] = merged['cust_email'].fillna('').str.extract(r'@(.+)$', expand=False).fillna('unknown')

# is_late
merged['is_late'] = merged['delivery_status'] == 'Delayed'

merged_df = merged.copy()

print('merged_df shape:', merged_df.shape)
merged_df.info()
merged_df.head()


#### Summary tables

In [ ]:
# Weekly revenue trends by region
weekly = merged_df.groupby(['order_week','region']).agg(
    weekly_revenue=('revenue','sum')
).reset_index()
pivot_week_region = weekly.pivot(index='order_week', columns='region', values='weekly_revenue').fillna(0)
display(pivot_week_region)

# Product category performance
cat_perf = merged_df.groupby('prod_category').agg(
    total_revenue=('revenue','sum'),
    total_quantity=('quantity','sum'),
    avg_discount=('discount_applied','mean')
).sort_values('total_revenue', ascending=False).reset_index()
display(cat_perf)

# Customer behaviour by loyalty tier and signup month
merged_df['signup_month'] = merged_df['cust_signup_date'].dt.to_period('M')
cust_behaviour = merged_df.groupby(['cust_loyalty_tier','signup_month']).agg(
    revenue=('revenue','sum'),
    orders=('order_id','nunique'),
    avg_order_value=('revenue','mean')
).reset_index()
display(cust_behaviour)

# Delivery performance by region and price band
delivery_perf = merged_df.groupby(['region','price_band'], observed=True).agg(
    total_orders=('order_id','count'),
    delayed_pct=('is_late',lambda x: 100*x.sum()/len(x))
).reset_index()
display(delivery_perf)

# Preferred payment methods by loyalty tier
pay_pref = merged_df.groupby(['cust_loyalty_tier','payment_method']).size().reset_index(name='count')
pay_pref = pay_pref.sort_values(['cust_loyalty_tier','count'], ascending=[True,False])
display(pay_pref)

#### Visual Exploration

In [ ]:
# 1. Line plot - weekly revenue trends by region
fig, ax = plt.subplots(figsize=(10,6))
for col in pivot_week_region.columns:
    ax.plot(pivot_week_region.index, pivot_week_region[col], marker='o', label=col)
ax.set_title('Weekly Revenue Trends by Region', fontsize=14)
ax.set_xlabel('ISO Week', fontsize=12)
ax.set_ylabel('Revenue (£)', fontsize=12)
ax.legend(title='Region')
ax.grid(True)
plt.tight_layout()
plt.show()


# 2. Bar chart - top 5 categories by revenue
top5 = cat_perf.head(5)
fig, ax = plt.subplots(figsize=(8,5))
sns.barplot(x='prod_category', y='total_revenue', hue='prod_category', data=top5, ax=ax, palette='viridis', dodge=False, legend=False)
ax.set_title('Top 5 Product Categories by Revenue', fontsize=14)
ax.set_ylabel('Revenue (£)', fontsize=12)
ax.set_xlabel('Category', fontsize=12)
for i, v in enumerate(top5['total_revenue']):
    ax.text(i, v + 10, f"{v:.0f}", ha='center')
plt.tight_layout()
plt.show()


# 3. Boxplot - quantity vs discount across categories
fig, ax = plt.subplots(figsize=(10,6))
sns.boxplot(x='prod_category', y='quantity', data=merged_df, ax=ax)
ax.set_title('Quantity Distribution by Product Category', fontsize=14)
ax.set_xlabel('Category', fontsize=12)
ax.set_ylabel('Quantity', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# 4. Heatmap - correlation between revenue, discount, quantity
corr_df = merged_df[['revenue','discount_applied','quantity']].corr()
fig, ax = plt.subplots(figsize=(5,4))
sns.heatmap(corr_df, annot=True, fmt=".2f", cmap='coolwarm', ax=ax)
ax.set_title('Correlation Matrix: Revenue, Discount, Quantity', fontsize=12)
plt.tight_layout()
plt.show()


# 5. Countplot - orders by loyalty tier with hue = region
fig, ax = plt.subplots(figsize=(8,5))
sns.countplot(x='cust_loyalty_tier', hue='region', data=merged_df, palette='Set2', ax=ax)
ax.set_title('Orders by Loyalty Tier and Region', fontsize=14)
ax.set_xlabel('Loyalty Tier', fontsize=12)
ax.set_ylabel('Number of Orders', fontsize=12)
plt.legend(title='Region')
plt.tight_layout()
plt.show()


# 6. Stacked bar - delivery status by price band
ds = merged_df.groupby(['price_band','delivery_status'], observed=True).size().unstack(fill_value=0)
ds.plot(kind='bar', stacked=True, figsize=(8,5), colormap='Pastel1')
plt.title('Delivery Status by Price Band', fontsize=14)
plt.xlabel('Price Band')
plt.ylabel('Number of Orders')
plt.xticks(rotation=0)
plt.legend(title='Delivery Status')
plt.tight_layout()
plt.show()

In [ ]:
merged_df['cust_signup_date'] = pd.to_datetime(merged_df['cust_signup_date'], errors='coerce')

merged_df['signup_month'] = merged_df['cust_signup_date'].dt.to_period('M').astype(str)

signup_tier = (
    merged_df.groupby(['signup_month', 'cust_loyalty_tier'], observed=True)
    .agg(total_revenue=('revenue', 'sum'))
    .reset_index()
)

display(signup_tier)

plt.figure(figsize=(10,6))
sns.lineplot(x='signup_month', y='total_revenue', hue='cust_loyalty_tier', data=signup_tier, marker='o')
plt.title('Revenue by Signup Month and Loyalty Tier')
plt.xlabel('Signup Month')
plt.ylabel('Total Revenue (£)')
plt.legend(title='Loyalty Tier')
plt.tight_layout()
plt.show()

#### Stretch Tasks

In [ ]:
q2_customers = merged_df.query(
    "'2025-04-01' <= cust_signup_date <= '2025-06-30'"
)['cust_customer_id'].unique()

q2_sales = merged_df[merged_df['cust_customer_id'].isin(q2_customers)].copy()
q2_sales['days_since_signup'] = (q2_sales['order_date'] - q2_sales['cust_signup_date']).dt.days

early_orders = q2_sales.query('0 <= days_since_signup <= 14')

high_discount_early = early_orders.query('discount_applied >= 0.20')

print(f"Q2 signups: {len(q2_customers)} customers")
print(f"Ordered in ≤14 days: {early_orders['cust_customer_id'].nunique()} customers")
print(f"With ≥20% discount: {high_discount_early.shape[0]} orders")

stretch_result = high_discount_early[['order_id', 'cust_customer_id', 'order_date', 
                                      'cust_signup_date', 'days_since_signup', 
                                      'discount_applied', 'revenue', 'prod_product_name']]
display(stretch_result)

In [ ]:
scaler = MinMaxScaler()

merged_df['revenue_norm'] = scaler.fit_transform(merged_df[['revenue']])
merged_df['unit_price_norm'] = scaler.fit_transform(merged_df[['unit_price']])

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(merged_df['revenue_norm'], kde=True, ax=axes[0], color='teal')
axes[0].set_title('Normalized Revenue Distribution')
sns.histplot(merged_df['unit_price_norm'], kde=True, ax=axes[1], color='coral')
axes[1].set_title('Normalized Unit Price Distribution')
plt.tight_layout()
plt.show()

In [ ]:
product_metrics = (
    merged_df.groupby('product_id')
    .agg(
        total_qty=('quantity', 'sum'),
        avg_discount=('discount_applied', 'mean'),
        delayed_rate=('is_late', 'mean'),
        total_orders=('order_id', 'count'),
        total_revenue=('revenue', 'sum')
    )
    .reset_index()
)

product_metrics = product_metrics.merge(
    products[['product_id', 'product_name', 'category']],
    on='product_id',
    how='left'
)

qty_threshold = product_metrics['total_qty'].quantile(0.25)
print(f"25th percentile quantity: {qty_threshold:.1f}")

product_metrics['low_quantity'] = product_metrics['total_qty'] < qty_threshold
product_metrics['high_discount'] = product_metrics['avg_discount'] > 0.15
product_metrics['high_delay'] = product_metrics['delayed_rate'] > 0.30

product_metrics['underperf_score'] = (
    product_metrics['low_quantity'].astype(int) +
    product_metrics['high_discount'].astype(int) +
    product_metrics['high_delay'].astype(int)
)

underperformers = product_metrics[product_metrics['underperf_score'] >= 2].copy()

print(f"\nUnderperforming products: {len(underperformers)}")
display(
    underperformers[['product_name', 'category', 'total_qty', 'avg_discount', 
                     'delayed_rate', 'underperf_score', 'total_revenue']]
    .sort_values('underperf_score', ascending=False)
)